In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import re
import os
import unicodedata
import numpy as np
import torch.optim as optim
from torchtext.datasets import TranslationDataset,Multi30k
from torchtext.data import Field,BucketIterator
device=torch.device('cuda:0')
import spacy
import random
import math
import time

In [2]:
SEED=1024

random.seed(SEED)
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic=True

In [3]:
spacy_de=spacy.load('de')
spacy_en=spacy.load('en')
def token_de(text):
    return [tok.text for tok in spacy_de.tokenizer(text)][::-1]
def token_en(text):
    return [tok.text for tok in spacy_en.tokenizer(text)]
SRC=Field(tokenize=token_de,init_token='<sos>',eos_token='<eos>',lower=True)
TRG=Field(tokenize=token_en,init_token='<sos>',eos_token='<eos>',lower=True)
train_data,valid_data,test_data=Multi30k.splits(exts=('.de','.en'),fields=(SRC,TRG))

print(f"Number of training examples: {len(train_data.examples)}")
print(f"Number of validation examples: {len(valid_data.examples)}")
print(f"Number of testing examples: {len(test_data.examples)}")
print(vars(train_data.examples[0]))

Number of training examples: 29000
Number of validation examples: 1014
Number of testing examples: 1000
{'src': ['.', 'büsche', 'vieler', 'nähe', 'der', 'in', 'freien', 'im', 'sind', 'männer', 'weiße', 'junge', 'zwei'], 'trg': ['two', 'young', ',', 'white', 'males', 'are', 'outside', 'near', 'many', 'bushes', '.']}


In [4]:
SRC.build_vocab(train_data,min_freq=2)
TRG.build_vocab(train_data,min_freq=2)
print(len(SRC.vocab))
print(len(TRG.vocab))
BATCH_SIZE=128
train_iter,valid_iter,test_iter=BucketIterator.splits((train_data,valid_data,test_data),batch_size=BATCH_SIZE,device=device)
print(train_iter)

7855
5893


In [5]:
class Encoder(nn.Module):
    def __init__(self,input_dim,emb_dim,hid_dim,n_layers,dropout):
        super().__init__()
        
        self.input_dim=input_dim
        self.emb_dim=emb_dim
        self.hid_dim=hid_dim
        self.n_layers=n_layers
        self.dropout=dropout
        self.embedding = nn.Embedding(input_dim, emb_dim)
        
        self.rnn = nn.LSTM(emb_dim, hid_dim, n_layers, dropout = dropout)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self,src):
        embedded=self.dropout(self.embedding(src))
        
        outputs,(hidden,cell)=self.rnn(embedded)
        
        return hidden,cell
class Decoder(nn.Module):
    def __init__(self,output_dim,emb_dim,hid_dim,n_layers,dropout):
        super().__init__()
        
        self.emb_dim=emb_dim
        self.hid_dim=hid_dim
        self.output_dim=output_dim
        self.n_layers=n_layers
        self.dropout=dropout
        
        self.embedding=nn.Embedding(output_dim,emb_dim)
        
        self.rnn=nn.LSTM(emb_dim,hid_dim,n_layers,dropout=dropout)
        
        self.out=nn.Linear(hid_dim,output_dim)
        
        self.dropout=nn.Dropout(dropout)
    def forward(self,inputs,hidden,cell):
        inputs=inputs.unsqueeze(0)
        embedded=self.dropout(self.embedding(inputs))
        
        output,(hidden,cell)=self.rnn(embedded,(hidden,cell))
        
        prediction=self.out(output.squeeze(0))
        
        return prediction,hidden,cell
class Seq2Seq(nn.Module):
    def __init__(self,encoder,decoder,device):
        super().__init__()
        
        self.encoder=encoder
        self.decoder=decoder
        self.device=device
        
        assert encoder.hid_dim==decoder.hid_dim,\
            "编码层和解码层隐藏层必须相同"
        assert encoder.n_layers==decoder.n_layers,\
            "编码器解码器层数必须相同"
    def forward(self,src,trg,teacher_forcing_ratio=0.5):
        
        batch_size=trg.shape[1]
        max_len=trg.shape[0]
        
        trg_vacab_size=self.decoder.output_dim
        
        outputs=torch.zeros(max_len,batch_size,trg_vacab_size).to(self.device)
        
        hidden,cell=self.encoder(src)
        
        inputs=trg[0,:]
        
        for t in range(1,max_len):
            output,hidden,cell=self.decoder(inputs,hidden,cell)
            outputs[t]=output
            teacher_forcing=random.random()<teacher_forcing_ratio
            top1=output.max(1)[1]
            inputs=(trg[t] if teacher_forcing else top1)
            
        return outputs

In [7]:
input_dim=len(SRC.vocab)
output_dim=len(TRG.vocab)

encode_emb_dim=256
decode_emb_dim=256

hid_dim=512
n_layers=2

enc_dropout=0.5
dec_dropout=0.5

enc=Encoder(input_dim,encode_emb_dim,hid_dim,n_layers,enc_dropout)
dec=Decoder(output_dim,decode_emb_dim,hid_dim,n_layers,dec_dropout)

model=Seq2Seq(enc,dec,device).to(device)
def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.uniform_(param.data, -0.08, 0.08)
        
model.apply(init_weights)

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(7855, 256)
    (rnn): LSTM(256, 512, num_layers=2, dropout=0.5)
    (dropout): Dropout(p=0.5)
  )
  (decoder): Decoder(
    (embedding): Embedding(5893, 256)
    (rnn): LSTM(256, 512, num_layers=2, dropout=0.5)
    (out): Linear(in_features=512, out_features=5893, bias=True)
    (dropout): Dropout(p=0.5)
  )
)

In [8]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')
optims=optim.Adam(model.parameters())
PAD_IDX = TRG.vocab.stoi['<pad>']

criterion = nn.CrossEntropyLoss(ignore_index = PAD_IDX)
def train(model, iterator, optimizer, criterion, clip):
    
    model.train()
    
    epoch_loss = 0
    
    for i, batch in enumerate(iterator):
        
        src = batch.src
        trg = batch.trg
        
        optimizer.zero_grad()
        
        output = model(src,trg)
        
        #trg = [trg sent len, batch size]
        #output = [trg sent len, batch size, output dim]
        
        output = output[1:].view(-1, output.shape[-1])
        trg = trg[1:].view(-1)
        
        #trg = [(trg sent len - 1) * batch size]
        #output = [(trg sent len - 1) * batch size, output dim]
        
        loss = criterion(output, trg)
        
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        optimizer.step()
        
        epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)
def evaluate(model, iterator, criterion):
    
    model.eval()
    
    epoch_loss = 0
    
    with torch.no_grad():
    
        for i, batch in enumerate(iterator):

            src = batch.src
            trg = batch.trg

            output = model(src, trg, 0) #turn off teacher forcing

            #trg = [trg sent len, batch size]
            #output = [trg sent len, batch size, output dim]

            output = output[1:].view(-1, output.shape[-1])
            trg = trg[1:].view(-1)

            #trg = [(trg sent len - 1) * batch size]
            #output = [(trg sent len - 1) * batch size, output dim]

            loss = criterion(output, trg)
            
            epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

The model has 13,899,013 trainable parameters


In [9]:
N_EPOCHS = 10
CLIP = 1

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss = train(model, train_iter, optims, criterion, CLIP)
    valid_loss = evaluate(model, valid_iter, criterion)
    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut1-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

Epoch: 01 | Time: 0m 29s
	Train Loss: 5.046 | Train PPL: 155.446
	 Val. Loss: 5.015 |  Val. PPL: 150.716
Epoch: 02 | Time: 0m 29s
	Train Loss: 4.512 | Train PPL:  91.067
	 Val. Loss: 4.809 |  Val. PPL: 122.615
Epoch: 03 | Time: 0m 29s
	Train Loss: 4.186 | Train PPL:  65.758
	 Val. Loss: 4.642 |  Val. PPL: 103.710
Epoch: 04 | Time: 0m 29s
	Train Loss: 3.971 | Train PPL:  53.046
	 Val. Loss: 4.399 |  Val. PPL:  81.355
Epoch: 05 | Time: 0m 29s
	Train Loss: 3.789 | Train PPL:  44.217
	 Val. Loss: 4.227 |  Val. PPL:  68.488
Epoch: 06 | Time: 0m 29s
	Train Loss: 3.606 | Train PPL:  36.817
	 Val. Loss: 4.169 |  Val. PPL:  64.629
Epoch: 07 | Time: 0m 29s
	Train Loss: 3.474 | Train PPL:  32.281
	 Val. Loss: 4.111 |  Val. PPL:  61.002
Epoch: 08 | Time: 0m 29s
	Train Loss: 3.327 | Train PPL:  27.858
	 Val. Loss: 3.961 |  Val. PPL:  52.484
Epoch: 09 | Time: 0m 29s
	Train Loss: 3.169 | Train PPL:  23.794
	 Val. Loss: 3.897 |  Val. PPL:  49.244
Epoch: 10 | Time: 0m 29s
	Train Loss: 3.068 | Train PPL